# Librerias

In [1]:
import pandas as pd 
import string
from nltk.tokenize import word_tokenize 
from string import punctuation 
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import svm
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import numpy as np

# Archivos

In [2]:
with open('archivos/stopwords.txt') as archivo_stopword:
    listaStopword=[line.rstrip('\n') for line in archivo_stopword]

# Funciones

## Filtrar StopWord

In [3]:
def Filtrar_StopWord(frase):
    frase = re.sub('[%s]' % re.escape(string.punctuation), ' ', frase)
    vector_frase = word_tokenize(frase)
    fraseFiltrada =""
    non_words = list(punctuation) 
    non_words.extend(['¿', '¡'])
    for palabra in vector_frase:
        if palabra.lower() not in lista_Stopword and palabra not in non_words:
            fraseFiltrada +=" "+palabra
    return fraseFiltrada

## Tokenize

In [4]:
def Tokenize(text):
        tokens = nltk.word_tokenize(text,  language='spanish')
        stems = []
        for item in tokens:
            stems.append(nltk.PorterStemmer().stem(item))
        return stems

## Procesar datos para Machine Learning 

In [5]:
def Procesar_DatosML(xtrain,xtest):
   vectorizer = TfidfVectorizer(tokenizer=Tokenize, max_df = 0.80, use_idf=True,min_df=1)
   #vectorizer    = CountVectorizer(min_df=1,binary=True) # metrica binaria
   vector_train  = vectorizer.fit_transform(xtrain)
   vector_test   = vectorizer.transform(xtest)
   #print(vectorizer.get_feature_names())
   return vector_train, vector_test, vectorizer

## Limpiar Corpus

In [6]:
def LimpiarCorpus(X_frases):
        for i in range(len(X_frases)):
            X_frases[i] = re.sub("[^a-zA-Z]", " ", X_frases[i].lower())
            X_frases[i] = ' '.join(X_frases[i].split())
        return X_frases

## Entrenar y medir modelos ML

In [7]:
def EntrenarMedirML(X_train, X_test, Y_train, Y_test,algoritmo):
    # MODELO
    if(algoritmo=="SVM"):
        modelo = svm.SVC(kernel='linear') 
        modelo.fit(X=X_train,y=Y_train)
        # PREDICCIÓN
        prediccion = modelo.predict(X_test)
    if(algoritmo=="Random Forest"):
        modelo=RandomForestClassifier(n_estimators=10)
        modelo.fit(X=X_train,y=Y_train)
        # PREDICCIÓN
        prediccion = modelo.predict(X_test)
    if(algoritmo=="Naive Bayes"):
        modelo = GaussianNB()
        modelo.fit(X=X_train,y=Y_train)
        # PREDICCIÓN
        prediccion = modelo.predict(X_test)
    # METRICAS
    #reporteMetricas= classification_report(Y_test, prediccion)
    return metrics.accuracy_score(Y_test, prediccion),modelo

# Enfoque 1 Analisis de sentimiento basico

In [8]:
def ClasificarPolaridadBase(corpus,algoritmo):
    X_frases = np.asarray(corpus[['A']])
    X_frases = X_frases.ravel()
    Y_sentimiento = np.asarray(corpus[['B']])
    Y_sentimiento = Y_sentimiento.ravel()
    X_frases = LimpiarCorpus(X_frases)
    # PARTIR CORPUS PARA ENTRENAR
    X_train, X_test, Y_train, Y_test = train_test_split(X_frases, Y_sentimiento, test_size=0.30)
    # TOKENIZACION + VECTORIZACIO
    X_train, X_test, vectorizer = Procesar_DatosML(X_train, X_test)
    #reporteMetricas= classification_report(Y_test, prediccion)
    return EntrenarMedirML(X_train, X_test, Y_train, Y_test,algoritmo)

# Main

## Clasificar el aspecto enfoque básico

In [9]:
corpus = pd.read_csv('archivos/aspectos.csv')
accuracyPromedio = 0
algoritmo="SVM"
numeroEjecuciones = 100
for i in range(numeroEjecuciones):
    accuracy,modelo = ClasificarPolaridadBase(corpus, algoritmo)
    accuracyPromedio +=accuracy
accuracyPromedio/numeroEjecuciones

0.813568773234201

## Analisis de sentimiento Enfoque: basico

In [11]:
corpus = pd.read_csv('archivos/sentimientos.csv')
accuracyPromedio = 0
algoritmo="SVM"
numeroEjecuciones = 100
for i in range(numeroEjecuciones):
    accuracy, kk = ClasificarPolaridadBase(corpus, algoritmo)
    accuracyPromedio += accuracy
accuracyPromedio/numeroEjecuciones

0.781569965870307